In [69]:
from cvxpy import *
import osqp
import numpy as np
import scipy as sp
from scipy import sparse
import sympy as sym

In [11]:
from time import perf_counter 

In [24]:
%matplotlib inline

In [70]:
# # components of position (meters)
o_x, o_y, o_z = sym.symbols("o_x, o_y, o_z")

# yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols("psi, theta, phi")

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols("v_x, v_y, v_z")

# components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols("w_x, w_y, w_z")

# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols("tau_x, tau_y, tau_z")

# net rotor force
f_z = sym.symbols("f_z")

In [71]:
m = sym.nsimplify(0.0315)  # mass of a Crazyflie drone

# Principle moments of inertia of a Crazyflie drone
J_x = sym.nsimplify(1.7572149113694408e-05)
J_y = sym.nsimplify(1.856979710568617e-05)
J_z = sym.nsimplify(2.7159794713754086e-05)

# Acceleration of gravity
g = 9.81

# Linear and angular velocity vectors (in body frame)
v_01in1 = sym.Matrix([[v_x], [v_y], [v_z]])
w_01in1 = sym.Matrix([[w_x], [w_y], [w_z]])

# Create moment of inertia matrix (in coordinates of the body frame).
J_in1 = sym.diag(J_x, J_y, J_z)

# Z-Y-X rotation sequence
Rz = sym.Matrix(
    [
        [sym.cos(psi), -sym.sin(psi), 0],
        [sym.sin(psi), sym.cos(psi), 0],
        [0, 0, 1],
    ]
)

Ry = sym.Matrix(
    [
        [sym.cos(theta), 0, sym.sin(theta)],
        [0, 1, 0],
        [-sym.sin(theta), 0, sym.cos(theta)],
    ]
)

Rx = sym.Matrix(
    [
        [1, 0, 0],
        [0, sym.cos(phi), -sym.sin(phi)],
        [0, sym.sin(phi), sym.cos(phi)],
    ]
)

R_1in0 = Rz * Ry * Rx

# Mapping from angular velocity to angular rates
# Compute the invserse of the mapping first:
Ninv = sym.Matrix.hstack(
    (Ry * Rx).T * sym.Matrix([[0], [0], [1]]),
    (Rx).T * sym.Matrix([[0], [1], [0]]),
    sym.Matrix([[1], [0], [0]]),
)
N = sym.simplify(Ninv.inv())  # this matrix N is what we actually want

# forces in world frame
f_in1 = R_1in0.T * sym.Matrix([[0], [0], [-m * g]]) + sym.Matrix(
    [[0], [0], [f_z]]
)

# torques in world frame
tau_in1 = sym.Matrix([[tau_x], [tau_y], [tau_z]])

In [121]:
# Full EOM:
f_sym = sym.Matrix.vstack(
    R_1in0 * v_01in1,
    N * w_01in1,
    (1 / m) * (f_in1 - w_01in1.cross(m * v_01in1)),
    J_in1.inv() * (tau_in1 - w_01in1.cross(J_in1 * w_01in1)),
)

x = sym.Matrix([o_x,o_y,o_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z])
u = sym.Matrix([tau_x,tau_y,tau_z,f_z])

A = f_sym.jacobian(x)
B = f_sym.jacobian(u)

In [125]:
# Discrete time model of a quadcopter
# The following two matrices are obtained after performing system 
# identification and state space linearization on the Crazyflie 2.0 quadcopter
# Solving the MPC problem with CVXPY:

[nx, nu] = B.shape

# Constraints

umin = np.array([-0.01, -0.01, -0.01, -0.01]) 
umax = np.array([2., 2., 2., 2.])
xmin = np.array([-np.inf,-np.inf,-np.inf,-np.pi/6,-np.pi/6,-np.pi/6,
                 -2.,-2.,-2.,-1.,-1.,-1.])
xmax = np.array([np.inf,np.inf,np.inf,np.pi/6,np.pi/6,np.pi/6,
                 20.,20.,20.,1.,1.,1.])
#x = [px, py, pz, psi, theta, psi, vx, vy, vz, wx, wy, wz]
#u = [tau_x, tau_y, tau_z, f_z]
# Objective function
Q = sparse.diags([10., 10., 10., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
QN = Q
R = 0.1*sparse.eye(4)

# Initial and reference states
x0 = np.zeros(12)
x0[2] = 0.5

xr = np.array([15., 15., 1.2, 0.,0.,0.,0.,0.,0.,0.,0.,0.])

# Prediction horizon
N = 50

# Define problem
u_ = Variable((nu, N))
x_ = Variable((nx, N+1))
x_init = Parameter(nx)

objective = 0
constraints = [x_[:,0] == x_init]
Ad = sym.lambdify((x,u),A)
Bd = sym.lambdify((x,u),B)

for k in range(N):
    objective += quad_form(x_[:,k] - xr, Q) + quad_form(u_[:,k], R)
    constraints += [x_[:,k+1] == Ad@x_[:,k] + Bd@u_[:,k]]
    constraints += [xmin <= x_[:,k], x_[:,k] <= xmax]
    constraints += [umin <= u_[:,k], u_[:,k] <= umax]
   

objective += quad_form(x_[:,N] - xr, QN)
prob = Problem(Minimize(objective), constraints)

# Simulate in closed loop
t0 = perf_counter()
nsim = 50

for i in range(nsim):
    x_init.value = x0
    
    prob.solve(solver=OSQP, warm_start=True)
    x0 = Ad.dot(x0) + Bd.dot(u_[:,0].value)
    print(x0)
print(perf_counter()-t0)

TypeError: float() argument must be a string or a real number, not 'function'